# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [12]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [13]:
# TODO: Import the necessary libs
# For example: 
# import os

# from lib.agents import Agent
# from lib.llm import LLM
# from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
# from lib.tooling import tool
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from tavily import TavilyClient
from dotenv import load_dotenv
from openai import OpenAI

In [14]:
# Load environment variables
load_dotenv()

# Validate required API keys with helpful error messages
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError(
        'OPENAI_API_KEY not found in environment variables. '
        'Please create a .env file with OPENAI_API_KEY="your_key"'
    )

tavily_api_key = os.getenv('TAVILY_API_KEY')
if not tavily_api_key:
    raise ValueError(
        'TAVILY_API_KEY not found in environment variables. '
        'Please create a .env file with TAVILY_API_KEY="your_key"'
    )

# Initialize clients with error handling
try:
    client = OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key=openai_api_key,
    )
except Exception as e:
    raise ValueError(f'Failed to initialize OpenAI client: {str(e)}')

try:
    tavily_client = TavilyClient(api_key=tavily_api_key)
except Exception as e:
    raise ValueError(f'Failed to initialize Tavily client: {str(e)}')

print('✅ API clients initialized successfully!')

In [15]:
from pydantic import BaseModel, Field
from typing import Optional

class AgentAnswer(BaseModel):
    answer: str = Field(description="The direct answer to the user's question.")
    source: str = Field(description="The source of the information, either the game's name or 'Web Search'.")
    fallback_used: bool = Field(description="True if a web search was required to answer the question.")
    internal_reasoning: str = Field(description="The agent's step-by-step reasoning for its conclusion.")


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [17]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

#  --- Tool 1: RAG Retrieval ---
def retrieve_game_info(query: str, n_results: int = 3) -> list[str]:
    """
    Queries the ChromaDB vector store to find relevant game information.
    """
    print(f"Tool: Retrieving game info for query: '{query}'")
    chroma_client = chromadb.PersistentClient(path="chromadb")

    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        api_base="https://openai.vocareum.com/v1",  # For Vocareum
        model_name="text-embedding-ada-002"
    )

    collection = chroma_client.get_collection(
        name="udaplay",
        embedding_function=embedding_fn
    )

    # --- This part was missing ---
    results = collection.query(query_texts=[query], n_results=n_results)

    # Format the retrieved documents for the next step
    retrieved_docs = []
    if results and results['metadatas'][0]:
        for meta in results['metadatas'][0]:
            # Pass the full JSON metadata as a string
            retrieved_docs.append(json.dumps(meta))

    return retrieved_docs


#### Evaluate Retrieval Tool

In [18]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
def evaluate_retrieval(query: str, context: list[str]) -> bool:
    """
    Uses an LLM to decide if the retrieved context is sufficient to answer the query.
    """
    # ... (rest of the function is the same) ...

    # A simple but effective prompt for the LLM
    prompt = f"""
    Based *only* on the provided context below, can you confidently answer the following user question?
    Respond with only "yes" or "no".

    User Question: "{query}"

    Context:
    ---
    {context}
    ---
    """

    # Use the configured client object here
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=5,
        temperature=0.0
    )

    decision = response.choices[0].message.content.strip().lower()
    print(f"Evaluation: LLM decision is '{decision}'.")
    return "yes" in decision



#### Game Web Search Tool

In [19]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.
def game_web_search(query: str) -> str:
    """
    Performs a web search using Tavily if the local DB is insufficient.
    """
    print(f"Tool: Performing web search for query: '{query}'")
    try:
        response = tavily_client.search(query=query, search_depth="basic")
        # We'll return the most relevant search result content
        return response['results'][0]['content']
    except Exception as e:
        print(f"Error during Tavily search: {e}")
        return "Web search failed."

### Agent

In [20]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
# %%
# In Udaplay_02_starter_project.ipynb

def run_udaplay_agent(user_query: str) -> AgentAnswer:
    """
    Runs the full agent workflow for a given user query.
    """
    reasoning = []

    # Step 1: Retrieve information from our local RAG pipeline
    retrieved_context = retrieve_game_info(user_query)
    reasoning.append(
        f"1. Searched local database for '{user_query}'. Found {len(retrieved_context)} potential matches.")

    # Step 2: Evaluate the retrieved information
    is_sufficient = evaluate_retrieval(user_query, retrieved_context)
    reasoning.append(f"2. Evaluated retrieved context. Decision: {'Sufficient' if is_sufficient else 'Insufficient'}.")

    final_context = ""
    source = ""
    fallback_used = False

    # Step 3: Decide whether to use the retrieved context or fall back to a web search
    if is_sufficient:
        # Use the local data
        final_context = "\n".join(retrieved_context)
        # For simplicity, we'll cite the first game found as the source
        source_game = json.loads(retrieved_context[0])
        source = source_game.get("Name", "Local Database")
        reasoning.append("3. Proceeding with information from local database.")
    else:
        # Fallback to web search
        fallback_used = True
        web_context = game_web_search(user_query)
        final_context = web_context
        source = "Web Search (Tavily)"
        reasoning.append("3. Local data was insufficient. Performing a web search.")
        reasoning.append("4. Web search provided new context.")

    # Step 4: Generate the final answer using the chosen context
    reasoning.append("5. Generating final answer based on the chosen context.")

    final_prompt = f"""
    You are the UdaPlay AI assistant. Your task is to answer the user's question based on the provided context.
    Be concise and direct in your answer.

    User Question: "{user_query}"

    Context to use:
    ---
    {final_context}
    ---
    """

    final_response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": final_prompt}]
    )

    answer = final_response.choices[0].message.content

    return AgentAnswer(
        answer=answer,
        source=source,
        fallback_used=fallback_used,
        internal_reasoning="\n".join(reasoning)
    )

In [21]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

# --- Test Query 1: Answer should be in the local DB ---
query1 = "When was God of War Ragnarok released?"
answer1 = run_udaplay_agent(query1)
print(json.dumps(answer1.model_dump(), indent=2))

# --- Test Query 2: Answer requires a web search ---
query2 = "What is Rockstar Games working on now?"
answer2 = run_udaplay_agent(query2)
print(json.dumps(answer2.model_dump(), indent=2))

# --- Test Query 3: Answer should be in the local DB ---
query3 = "What platform was Pokémon Red launched on?"
answer3 = run_udaplay_agent(query3)
print(json.dumps(answer3.model_dump(), indent=2))


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


Tool: Retrieving game info for query: 'When was God of War Ragnarok released?'
Evaluation: LLM decision is 'no'.
Tool: Performing web search for query: 'When was God of War Ragnarok released?'


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


{
  "answer": "God of War Ragnarok was released on November 9, 2022, on PS5 and PS4.",
  "source": "Web Search (Tavily)",
  "fallback_used": true,
  "internal_reasoning": "1. Searched local database for 'When was God of War Ragnarok released?'. Found 3 potential matches.\n2. Evaluated retrieved context. Decision: Insufficient.\n3. Local data was insufficient. Performing a web search.\n4. Web search provided new context.\n5. Generating final answer based on the chosen context."
}
Tool: Retrieving game info for query: 'What is Rockstar Games working on now?'
Evaluation: LLM decision is 'no'.
Tool: Performing web search for query: 'What is Rockstar Games working on now?'


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


{
  "answer": "Rockstar Games is working on \"Strange Tales of the West.\"",
  "source": "Web Search (Tavily)",
  "fallback_used": true,
  "internal_reasoning": "1. Searched local database for 'What is Rockstar Games working on now?'. Found 3 potential matches.\n2. Evaluated retrieved context. Decision: Insufficient.\n3. Local data was insufficient. Performing a web search.\n4. Web search provided new context.\n5. Generating final answer based on the chosen context."
}
Tool: Retrieving game info for query: 'What platform was Pokémon Red launched on?'
Evaluation: LLM decision is 'no'.
Tool: Performing web search for query: 'What platform was Pokémon Red launched on?'
{
  "answer": "Pok\u00e9mon Red was launched on the Game Boy platform.",
  "source": "Web Search (Tavily)",
  "fallback_used": true,
  "internal_reasoning": "1. Searched local database for 'What platform was Pok\u00e9mon Red launched on?'. Found 3 potential matches.\n2. Evaluated retrieved context. Decision: Insufficient.\n

### (Optional) Advanced

In [22]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes